In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from flask import Flask, render_template, redirect
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
import geopandas


In [23]:
# Make a reference to the books.csv file path
trulia_path = "Resources/trulia_data.csv"
# Import the books.csv file as a DataFrame
trulia_cvs = pd.read_csv(trulia_path)

In [24]:
#View all the columns
pd.set_option('display.max_columns', None)
trulia_cvs.head()

,Uniq Id,Crawl Timestamp,Url,Title,Description,Price,Image Url,Style,Sqr Ft,Longitude,Latitude,Home Id,Lot Size,Beds,Bath,Year Built,Price Sqr Ft,Features,Last Sold Year,Last Sold For,Last Tax Assestment,Last Tax Year,Address,City,State,Zipcode,Property Type,Address Full,Facts,Days On Trulia,Listing Agent Name,Listing Agent Contact No,Agent Name 1,Agent Contact No 1,Agent Name 2,Agent Contact No 2,Agent Name 3,Agent Contact No 3,Agent Name 4,Agent Contact No 4,Agent Name 5,Agent Contact No 5,Brokername,Image 1,Image 2,Image 3,Image 4,Image 5,Image 6,Image 7,Image 8,Image 9,Image 10,Image 11,Image 12,Image 13,Image 14,Image 15,Image 16,Image 17,Image 18,Image 19,Image 20,Image 21,Image 22,Image 23,Image 24,Image 25
0,d6521bac40600f17287f0a5a8b3efc8d,2019-09-19 05:40:24 +0000,https://www.trulia.com/p/az/phoenix/630-w-echo...,"630 W Echo Ln Phoenix, AZ 85021",NEW CONSTRUCTION in the North Central Corrido...,"$895,900",https://static.trulia-cdn.com/pictures/thumbs_...,4 Beds / 4.5 Baths,"3,447 sqft",-112.081985,33.560055,3.259497e+09,"7,895 sqft",4.0,4.5,2019.0,$260/sqft,"Single Family Home | $260/sqft | Lot Size: 7,8...",NaN,NaN,"$27,645",2018.0,"630 W Echo Ln Phoenix, AZ 85021",Phoenix,AZ,85021.0,NaN,"630 W Echo Ln, Phoenix, AZ 85021","Lot Land | 7,895 sqft lot size | County: Maric...",110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,659d62537d940a76e3349bd39a72bd92,2019-09-19 03:48:41 +0000,https://www.trulia.com/p/tx/dallas/2367-highwo...,"2367 Highwood Dr Dallas, TX 75228",UPDATED EAST DALLAS HOME READY FOR MOVE-IN. H...,"$247,000",https://static.trulia-cdn.com/pictures/thumbs_...,3 Beds / 2 Baths,"1,767 sqft",-96.676250,32.829227,1.067905e+09,"7,877 sqft",3.0,2.0,1954.0,$140/sqft,"Single Family Home | $140/sqft | Lot Size: 7,8...",NaN,NaN,NaN,NaN,"2367 Highwood Dr Dallas, TX 75228",Dallas,TX,75228.0,NaN,"2367 Highwood Dr, Dallas, TX 75228","3 Beds | 2 Baths | 1,767 sqft | Single Family ...",126.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...
2,a003f7017e34b3c8486ffd54cecf9f0d,2019-09-19 09:33:07 +0000,https://www.trulia.com/p/ny/buffalo/30-hurlock...,"30 Hurlock Ave Buffalo, NY 14211",This single-family home is located at 30 Hurl...,

In [25]:
trulia_cvs.groupby('City')['Uniq Id'].count().sort_values()

City
Queen Creek             1
Richardson              1
Greenwell Springs       1
Red Oak                 1
Corp Christi            1
                     ... 
Miami                 821
Las Vegas             944
San Antonio           951
Chicago               985
Houston              1077
Name: Uniq Id, Length: 676, dtype: int64

In [26]:
#Show all dtypes
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(trulia_cvs.dtypes)


Uniq Id                      object
Crawl Timestamp              object
Url                          object
Title                        object
Description                  object
Price                        object
Image Url                    object
Style                        object
Sqr Ft                       object
Longitude                   float64
Latitude                    float64
Home Id                     float64
Lot Size                     object
Beds                        float64
Bath                        float64
Year Built                  float64
Price Sqr Ft                 object
Features                     object
Last Sold Year              float64
Last Sold For                object
Last Tax Assestment          object
Last Tax Year               float64
Address                      object
City                         object
State                        object
Zipcode                     float64
Property Type                object
Address Full                

In [27]:
#Cleaning Up Currency Data with Pandas
trulia_cvs['Price'] = trulia_cvs['Price'].replace({'\$': '', ',': '', '\+':'', 'Contact For Estimate':"NaN", '/mo':'', 'Contact For Price':"0"}, regex=True)
trulia_cvs['Price']

0         895900
1         247000
2          44900
3         959000
4          83500
          ...   
30001     300000
30002     119000
30003     210000
30004     329000
30005    4500000
Name: Price, Length: 30006, dtype: object

In [28]:
with pd.option_context('display.max_rows', None):
    print(trulia_cvs['Price'])


0          895900
1          247000
2           44900
3          959000
4           83500
5         1295000
6          235288
7          129900
8           62900
9          546416
10         139900
11         749900
12         238956
13        1449900
14         369000
15         279900
16          95000
17        1300000
18         350000
19         375590
20         139900
21         402100
22         206980
23         149900
24         230000
25         289900
26         799000
27         171900
28         207000
29         272471
30            NaN
31         735000
32         155000
33         415000
34         164900
35         390000
36         999900
37          73700
38         109000
39         415000
40         385000
41        2775000
42         265000
43         150000
44         539990
45         445000
46         679000
47         300825
48         239000
49         775000
50         299280
51         534000
52         399000
53         314900
54         415000
55        

In [29]:
# Use pd.to_numeric() method to convert the datatype of the Amount column
trulia_cvs['Price']= trulia_cvs['Price'].astype("float")

In [31]:
trulia_cvs['Price'].describe()

count    2.978000e+04
mean     5.297895e+05
std      1.141204e+06
min      0.000000e+00
25%      1.850000e+05
50%      3.099000e+05
75%      5.345000e+05
max      6.000000e+07
Name: Price, dtype: float64

In [32]:
pd.set_option('display.max_columns', None)
trulia_cvs.head()

,Uniq Id,Crawl Timestamp,Url,Title,Description,Price,Image Url,Style,Sqr Ft,Longitude,Latitude,Home Id,Lot Size,Beds,Bath,Year Built,Price Sqr Ft,Features,Last Sold Year,Last Sold For,Last Tax Assestment,Last Tax Year,Address,City,State,Zipcode,Property Type,Address Full,Facts,Days On Trulia,Listing Agent Name,Listing Agent Contact No,Agent Name 1,Agent Contact No 1,Agent Name 2,Agent Contact No 2,Agent Name 3,Agent Contact No 3,Agent Name 4,Agent Contact No 4,Agent Name 5,Agent Contact No 5,Brokername,Image 1,Image 2,Image 3,Image 4,Image 5,Image 6,Image 7,Image 8,Image 9,Image 10,Image 11,Image 12,Image 13,Image 14,Image 15,Image 16,Image 17,Image 18,Image 19,Image 20,Image 21,Image 22,Image 23,Image 24,Image 25
0,d6521bac40600f17287f0a5a8b3efc8d,2019-09-19 05:40:24 +0000,https://www.trulia.com/p/az/phoenix/630-w-echo...,"630 W Echo Ln Phoenix, AZ 85021",NEW CONSTRUCTION in the North Central Corrido...,895900.0,https://static.trulia-cdn.com/pictures/thumbs_...,4 Beds / 4.5 Baths,"3,447 sqft",-112.081985,33.560055,3.259497e+09,"7,895 sqft",4.0,4.5,2019.0,$260/sqft,"Single Family Home | $260/sqft | Lot Size: 7,8...",NaN,NaN,"$27,645",2018.0,"630 W Echo Ln Phoenix, AZ 85021",Phoenix,AZ,85021.0,NaN,"630 W Echo Ln, Phoenix, AZ 85021","Lot Land | 7,895 sqft lot size | County: Maric...",110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,659d62537d940a76e3349bd39a72bd92,2019-09-19 03:48:41 +0000,https://www.trulia.com/p/tx/dallas/2367-highwo...,"2367 Highwood Dr Dallas, TX 75228",UPDATED EAST DALLAS HOME READY FOR MOVE-IN. H...,247000.0,https://static.trulia-cdn.com/pictures/thumbs_...,3 Beds / 2 Baths,"1,767 sqft",-96.676250,32.829227,1.067905e+09,"7,877 sqft",3.0,2.0,1954.0,$140/sqft,"Single Family Home | $140/sqft | Lot Size: 7,8...",NaN,NaN,NaN,NaN,"2367 Highwood Dr Dallas, TX 75228",Dallas,TX,75228.0,NaN,"2367 Highwood Dr, Dallas, TX 75228","3 Beds | 2 Baths | 1,767 sqft | Single Family ...",126.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...
2,a003f7017e34b3c8486ffd54cecf9f0d,2019-09-19 09:33:07 +0000,https://www.trulia.com/p/ny/buffalo/30-hurlock...,"30 Hurlock Ave Buffalo, NY 14211",This single-family home is located at 30 Hurl...,4490

In [33]:
dfr=trulia_cvs.rename(columns={"Sqr Ft":"sqr_ft",
                       "Year Built":"year",
                        "Price":"price"})
dfr

,Uniq Id,Crawl Timestamp,Url,Title,Description,price,Image Url,Style,sqr_ft,Longitude,Latitude,Home Id,Lot Size,Beds,Bath,year,Price Sqr Ft,Features,Last Sold Year,Last Sold For,Last Tax Assestment,Last Tax Year,Address,City,State,Zipcode,Property Type,Address Full,Facts,Days On Trulia,Listing Agent Name,Listing Agent Contact No,Agent Name 1,Agent Contact No 1,Agent Name 2,Agent Contact No 2,Agent Name 3,Agent Contact No 3,Agent Name 4,Agent Contact No 4,Agent Name 5,Agent Contact No 5,Brokername,Image 1,Image 2,Image 3,Image 4,Image 5,Image 6,Image 7,Image 8,Image 9,Image 10,Image 11,Image 12,Image 13,Image 14,Image 15,Image 16,Image 17,Image 18,Image 19,Image 20,Image 21,Image 22,Image 23,Image 24,Image 25
0,d6521bac40600f17287f0a5a8b3efc8d,2019-09-19 05:40:24 +0000,https://www.trulia.com/p/az/phoenix/630-w-echo...,"630 W Echo Ln Phoenix, AZ 85021",NEW CONSTRUCTION in the North Central Corrido...,895900.0,https://static.trulia-cdn.com/pictures/thumbs_...,4 Beds / 4.5 Baths,"3,447 sqft",-112.081985,33.560055,3.259497e+09,"7,895 sqft",4.0,4.5,2019.0,$260/sqft,"Single Family Home | $260/sqft | Lot Size: 7,8...",NaN,NaN,"$27,645",2018.0,"630 W Echo Ln Phoenix, AZ 85021",Phoenix,AZ,85021.0,NaN,"630 W Echo Ln, Phoenix, AZ 85021","Lot Land | 7,895 sqft lot size | County: Maric...",110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,659d62537d940a76e3349bd39a72bd92,2019-09-19 03:48:41 +0000,https://www.trulia.com/p/tx/dallas/2367-highwo...,"2367 Highwood Dr Dallas, TX 75228",UPDATED EAST DALLAS HOME READY FOR MOVE-IN. H...,247000.0,https://static.trulia-cdn.com/pictures/thumbs_...,3 Beds / 2 Baths,"1,767 sqft",-96.676250,32.829227,1.067905e+09,"7,877 sqft",3.0,2.0,1954.0,$140/sqft,"Single Family Home | $140/sqft | Lot Size: 7,8...",NaN,NaN,NaN,NaN,"2367 Highwood Dr Dallas, TX 75228",Dallas,TX,75228.0,NaN,"2367 Highwood Dr, Dallas, TX 75228","3 Beds | 2 Baths | 1,767 sqft | Single Family ...",126.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...,https://static.trulia-cdn.com/pictures/thumbs_...
2,a003f7017e34b3c8486ffd54cecf9f0d,2019-09-19 09:33:07 +0000,https://www.trulia.com/p/ny/buffalo/30-hurlock...,"30 Hurlock Ave Buffalo, NY 14211",This single-family home is located at 30 Hurl...,44900.0,ht

In [34]:
trulia_cvs['Sqr Ft'] = trulia_cvs['Sqr Ft'].replace({'\ sqft': '', ',': '', 'NaN':"0", '/mo':'', 'Contact For Price':"0"}, regex=True)
trulia_cvs['Sqr Ft']

0         3447
1         1767
2         1232
3         1417
4          440
         ...  
30001      733
30002     1716
30003      NaN
30004      NaN
30005    10149
Name: Sqr Ft, Length: 30006, dtype: object

In [35]:
df = trulia_cvs.dropna(how='any')
df.count()

Uniq Id            0
Crawl Timestamp    0
Url                0
Title              0
Description        0
                  ..
Image 21           0
Image 22           0
Image 23           0
Image 24           0
Image 25           0
Length: 68, dtype: int64

In [36]:
#Build data frame
price=trulia_cvs["Price"]
price

sqr_ft=trulia_cvs["Sqr Ft"]
sqr_ft

beds=trulia_cvs["Beds"]
beds

bath=trulia_cvs["Bath"]
bath

city=trulia_cvs["City"]
city

state=trulia_cvs["State"]
state

0        AZ
1        TX
2        NY
3        NY
4        FL
         ..
30001    FL
30002    CA
30003    LA
30004    NY
30005    NC
Name: State, Length: 30006, dtype: object

In [39]:
df=pd.DataFrame({"price":price,
                 "sqrft":sqr_ft,
                 "beds":beds,
                 "bath":bath,
                 "city":city,
                 "state":state})
df

,price,sqrft,beds,bath,city,state
0,895900.0,3447,4.0,4.5,Phoenix,AZ
1,247000.0,1767,3.0,2.0,Dallas,TX
2,44900.0,1232,3.0,1.0,Buffalo,NY
3,959000.0,1417,3.0,2.0,Flushing,NY
4,83500.0,440,NaN,1.0,Miami Gardens,FL
...,...,...,...,...,...,...
30001,300000.0,733,1.0,1.0,Miami,FL
30002,119000.0,1716,3.0,2.0,Modesto,CA
30003,210000.0,NaN,NaN,NaN,Baton Rouge,LA
30004,329000.0,NaN,7.0,3.0,Brooklyn,NY


In [40]:
df=df.dropna(how='any')
df.count()

price    26696
sqrft    26696
beds     26696
bath     26696
city     26696
state    26696
dtype: int64

In [41]:
df

,price,sqrft,beds,bath,city,state
0,895900.0,3447,4.0,4.5,Phoenix,AZ
1,247000.0,1767,3.0,2.0,Dallas,TX
2,44900.0,1232,3.0,1.0,Buffalo,NY
3,959000.0,1417,3.0,2.0,Flushing,NY
5,1295000.0,3814,4.0,4.0,Scottsdale,AZ
...,...,...,...,...,...,...
29999,171000.0,1269,3.0,2.0,Fort Worth,TX
30000,409990.0,1720,3.0,2.5,Philadelphia,PA
30001,300000.0,733,1.0,1.0,Miami,FL
30002,119000.0,1716,3.0,2.0,Modesto,CA


In [42]:
with pd.option_context('display.max_rows', None):
    print(df['price'])

0          895900.0
1          247000.0
2           44900.0
3          959000.0
5         1295000.0
6          235288.0
7          129900.0
8           62900.0
9          546416.0
10         139900.0
11         749900.0
12         238956.0
13        1449900.0
14         369000.0
15         279900.0
16          95000.0
17        1300000.0
18         350000.0
19         375590.0
20         139900.0
21         402100.0
22         206980.0
23         149900.0
24         230000.0
25         289900.0
26         799000.0
27         171900.0
28         207000.0
29         272471.0
31         735000.0
32         155000.0
33         415000.0
34         164900.0
35         390000.0
36         999900.0
37          73700.0
38         109000.0
39         415000.0
40         385000.0
41        2775000.0
42         265000.0
43         150000.0
44         539990.0
45         445000.0
46         679000.0
47         300825.0
48         239000.0
49         775000.0
50         299280.0
52         399000.0


In [43]:
# Export file as a CSV, without the Pandas index, but with the header
df.to_csv("../scrap_mcubicos/democ.csv", index=False, header=True)